# Red Project AI Context Viewer & Script Manager

This notebook helps you view system status information and manage the health check system for your Red project. It provides an interface to:

1. Make the health check script executable
2. Run health checks on demand
3. View system status and AI context
4. Create script wrappers for tracking provisioning script execution
5. Generate on-start scripts for Vast.ai

In [ ]:
# Standard imports for notebook functionality
import os
import sys
import time
from IPython.display import Markdown, display

# Check if running in Vast.ai environment
in_vastai = os.path.exists('/workspace')
if in_vastai:
    workspace_dir = '/workspace'
    logs_dir = '/logs'
else:
    # Local development environment fallback
    workspace_dir = os.getcwd()
    logs_dir = os.path.join(workspace_dir, 'logs')
    os.makedirs(logs_dir, exist_ok=True)
    
print(f"Workspace directory: {workspace_dir}")
print(f"Logs directory: {logs_dir}")

## Setup Health Check Script

The first step is to make sure the health check script exists and is executable.

In [ ]:
# Make the health check script executable
health_check_path = os.path.join(workspace_dir, 'health_check.sh')

if not os.path.exists(health_check_path):
    print(f"⚠️ Health check script not found at {health_check_path}")
    print("Creating a new script file...")
    !touch {health_check_path}
    print("Please edit the script content or re-run the notebook setup cell.")
else:
    print(f"✅ Health check script found at {health_check_path}")
    
# Make it executable
!chmod +x {health_check_path}
print("✅ Made health_check.sh executable")

In [ ]:
# Run the health check script to update the context
print("Running health check...")
!bash {health_check_path}
print("✅ Health check completed")

## AI Context Display

This section displays the current AI context information for easy viewing.

In [ ]:
# Display the AI context file
context_path = os.path.join(workspace_dir, 'red_ai_context.md')

if os.path.exists(context_path):
    with open(context_path, 'r') as f:
        content = f.read()
    display(Markdown(content))
else:
    print(f"⚠️ Context file not found at {context_path}")
    print("Run the health check script first to generate it.")

## VisoMaster Path Status

This section displays the current state of your VisoMaster installation and checks for path consistency.

In [ ]:
# Detect VisoMaster locations and check paths
locations = [
    '/VisoMaster',
    '/workspace/VisoMaster',
    '/workspace/VisoMaster-main',
    '/root/VisoMaster'
]

print("🔍 Checking for VisoMaster installations...\n")
found = False

# Check each location
for location in locations:
    if os.path.exists(location):
        found = True
        print(f"✅ Found VisoMaster at: {location}")
        
        # Check required directories
        model_assets = os.path.join(location, 'model_assets')
        models = os.path.join(location, 'models')
        
        if os.path.exists(model_assets):
            asset_count = len([f for f in os.listdir(model_assets) if f.endswith('.onnx')])
            print(f"   - model_assets: ✅ (Contains {asset_count} .onnx files)")
        else:
            print(f"   - model_assets: ❌ (Directory missing!)")
            
        if os.path.exists(models):
            model_count = len([f for f in os.listdir(models) if f.endswith('.onnx')])
            print(f"   - models: ✅ (Contains {model_count} .onnx files)")
        else:
            print(f"   - models: ❌ (Directory missing!)")
            
        print("")

if not found:
    print("❌ VisoMaster not found in any standard location!")
    print("   Please check your installation and provisioning scripts.")

## Provisioning Script Logging

This section creates wrapper scripts for the provisioning scripts to track their execution separately.

In [ ]:
# Create wrapper for vast_ai_provisioning_script.sh
vast_script_path = "/vast_ai_provisioning_script.sh"
vast_wrapper_path = "/vast_ai_provisioning_script_wrapper.sh"

if os.path.exists(vast_script_path):
    with open(vast_wrapper_path, "w") as f:
        f.write(f"""#!/bin/bash
mkdir -p {logs_dir}
echo \"Starting vast_ai_provisioning_script.sh at $(date)\" > {logs_dir}/vast_ai_provisioning.log
bash {vast_script_path} 2>&1 | tee -a {logs_dir}/vast_ai_provisioning.log
echo \"vast_ai_provisioning_script.sh completed at $(date)\" >> {logs_dir}/vast_ai_provisioning.log
""")
    !chmod +x {vast_wrapper_path}
    print(f"✅ Created wrapper for {vast_script_path}")
else:
    print(f"⚠️ Script not found at {vast_script_path}")

In [ ]:
# Create wrapper for src/provisioning_script.sh
src_script_path = "/src/provisioning_script.sh"
src_wrapper_path = "/src/provisioning_script_wrapper.sh"

if os.path.exists(src_script_path):
    with open(src_wrapper_path, "w") as f:
        f.write(f"""#!/bin/bash
mkdir -p {logs_dir}
echo \"Starting src/provisioning_script.sh at $(date)\" > {logs_dir}/src_provisioning.log
bash {src_script_path} 2>&1 | tee -a {logs_dir}/src_provisioning.log
echo \"src/provisioning_script.sh completed at $(date)\" >> {logs_dir}/src_provisioning.log
""")
    !chmod +x {src_wrapper_path}
    print(f"✅ Created wrapper for {src_script_path}")
else:
    print(f"⚠️ Script not found at {src_script_path}")

In [ ]:
# Create wrapper for VisoMaster provisioning_script.sh
viso_script_path = "/VisoMaster/provisioning_script.sh"
viso_wrapper_path = "/VisoMaster/provisioning_script_wrapper.sh"

if os.path.exists(viso_script_path):
    with open(viso_wrapper_path, "w") as f:
        f.write(f"""#!/bin/bash
mkdir -p {logs_dir}
echo \"Starting VisoMaster provisioning_script.sh at $(date)\" > {logs_dir}/viso_provisioning.log
cd /VisoMaster  # Change to VisoMaster directory before running
bash {viso_script_path} 2>&1 | tee -a {logs_dir}/viso_provisioning.log
echo \"VisoMaster provisioning_script.sh completed at $(date)\" >> {logs_dir}/viso_provisioning.log
""")
    !chmod +x {viso_wrapper_path}
    print(f"✅ Created wrapper for {viso_script_path}")
else:
    print(f"⚠️ Script not found at {viso_script_path}")

## Model Download Helper

This cell creates a helper script to ensure model downloads work correctly.

In [ ]:
# Create a model download helper script
model_helper_path = os.path.join(workspace_dir, "model_download_helper.sh")

with open(model_helper_path, "w") as f:
    f.write("""#!/bin/bash
# Model download helper script that ensures paths are correct

# Print debug info
echo "Current directory before CD: $(pwd)"

# Check all possible VisoMaster locations
VISOMASTER_PATH=""

if [ -d "/VisoMaster" ]; then
    VISOMASTER_PATH="/VisoMaster"
elif [ -d "/workspace/VisoMaster" ]; then 
    VISOMASTER_PATH="/workspace/VisoMaster"
elif [ -d "/workspace/VisoMaster-main" ]; then
    VISOMASTER_PATH="/workspace/VisoMaster-main"
elif [ -d "./VisoMaster" ]; then
    VISOMASTER_PATH="./VisoMaster"
    VISOMASTER_PATH=$(realpath $VISOMASTER_PATH)
elif [ -d "./VisoMaster-main" ]; then
    VISOMASTER_PATH="./VisoMaster-main"
    VISOMASTER_PATH=$(realpath $VISOMASTER_PATH)
fi

# If VisoMaster is found
if [ -n "$VISOMASTER_PATH" ]; then
    echo "VisoMaster found at: $VISOMASTER_PATH"
    
    # Change to the VisoMaster directory
    cd "$VISOMASTER_PATH"
    echo "Current directory after CD: $(pwd)"
    
    # Check if model_assets directory exists, if not create it
    if [ ! -d "./model_assets" ]; then
        echo "Creating model_assets directory"
        mkdir -p "./model_assets"
    fi
    
    # Check if models directory exists, if not create it
    if [ ! -d "./models" ]; then
        echo "Creating models directory"
        mkdir -p "./models"
    fi
    
    # Run the download script from the VisoMaster directory
    echo "Running download_models.py..."
    python download_models.py
    
    echo "Download script execution complete"
    
    # Ensure models are linked properly
    echo "Creating symlinks from model_assets to models..."
    find ./model_assets -type f -name "*.onnx" -exec ln -sf {} ./models/ \;
    
    # List the model files to verify they were downloaded
    echo "Contents of model_assets directory:"
    ls -la ./model_assets/
    
    echo "Contents of models directory:"
    ls -la ./models/
    
    exit 0
else
    echo "ERROR: Could not find VisoMaster directory!"
    echo "Current working directory: $(pwd)"
    echo "Directory listing:"
    ls -la
    
    exit 1
fi
""")

!chmod +x {model_helper_path}
print(f"✅ Created model download helper at {model_helper_path}")

## On-Start Script Creator

This cell creates a file with the fixed on-start script that you can copy to the Vast.ai interface

In [ ]:
# Create the fixed on-start script file
onstart_path = os.path.join(workspace_dir, "fixed_onstart_script.sh")
improved_onstart_path = os.path.join(workspace_dir, "improved_onstart.sh")

# Standard version (no separate logging)
with open(onstart_path, "w") as f:
    f.write("""#!/bin/bash
if [ -n "$PROVISIONING_SCRIPT" ]; then
  curl -s -o /tmp/prov.sh "$PROVISIONING_SCRIPT" || wget -q -O /tmp/prov.sh "$PROVISIONING_SCRIPT"
  chmod +x /tmp/prov.sh && bash /tmp/prov.sh
fi
for s in /vast_ai_provisioning_script.sh /src/provisioning_script.sh /VisoMaster/provisioning_script.sh; do
  if [ -f "$s" ]; then echo "Running: $s"; bash "$s"; fi
done
""")

# Improved version with separate logging
with open(improved_onstart_path, "w") as f:
    f.write("""#!/bin/bash
# Improved on-start script with separate logging for each provisioning script

mkdir -p /logs
echo "Starting improved on-start script at $(date)" > /logs/onstart.log

# Run the provisioning script from environment variable if specified
if [ -n "$PROVISIONING_SCRIPT" ]; then
  echo "Using PROVISIONING_SCRIPT: $PROVISIONING_SCRIPT" | tee -a /logs/onstart.log
  curl -s -o /tmp/prov.sh "$PROVISIONING_SCRIPT" || wget -q -O /tmp/prov.sh "$PROVISIONING_SCRIPT"
  chmod +x /tmp/prov.sh
  echo "Starting PROVISIONING_SCRIPT at $(date)" > /logs/env_provisioning.log
  bash /tmp/prov.sh 2>&1 | tee -a /logs/env_provisioning.log
  echo "PROVISIONING_SCRIPT completed at $(date)" >> /logs/env_provisioning.log
fi

# Run each local provisioning script with separate logging
for script_path in /vast_ai_provisioning_script.sh /src/provisioning_script.sh /VisoMaster/provisioning_script.sh; do
  if [ -f "$script_path" ]; then
    script_name=$(basename "$script_path" .sh)
    log_file="/logs/${script_name}.log"
    
    echo "Running: $script_path with log at $log_file" | tee -a /logs/onstart.log
    echo "Starting $script_path at $(date)" > "$log_file"
    bash "$script_path" 2>&1 | tee -a "$log_file"
    echo "$script_path completed at $(date)" >> "$log_file"
  fi
done

# Ensure model directories exist and are properly set up
if [ -d "/VisoMaster" ]; then
  mkdir -p /VisoMaster/model_assets
  mkdir -p /VisoMaster/models
  
  # Create symlinks from model_assets to models if needed
  if [ -d "/VisoMaster/model_assets" ] && [ "$(ls -A /VisoMaster/model_assets 2>/dev/null)" ]; then
    echo "Creating symlinks from model_assets to models directory..." | tee -a /logs/onstart.log
    find /VisoMaster/model_assets -type f -name "*.onnx" -exec ln -sf {} /VisoMaster/models/ \;
  fi
fi

echo "Completed on-start script at $(date)" >> /logs/onstart.log
""")

!chmod +x {onstart_path}
!chmod +x {improved_onstart_path}

print("Fixed On-Start Script (copy this to Vast.ai):")
print("")
with open(improved_onstart_path, "r") as f:
    print(f.read())

## Copy Context to Share with AI

Use this cell to view and copy the raw content of the AI context file to share with Claude or other AI assistants.

In [ ]:
# View the raw content of red_ai_context.md (for copying)
context_path = os.path.join(workspace_dir, 'red_ai_context.md')

if os.path.exists(context_path):
    with open(context_path, 'r') as f:
        content = f.read()
    print(content)
else:
    print(f"⚠️ Context file not found at {context_path}")
    print("Run the health check script first to generate it.")

## Setup Cron Job

This cell will set up a cron job to run the health check script regularly.

In [ ]:
# Check if cron job already exists
try:
    existing_cron = !crontab -l 2>/dev/null
    has_health_check = any('health_check.sh' in line for line in existing_cron)
    
    if has_health_check:
        print("✅ Cron job for health check already exists:")
        for line in existing_cron:
            if 'health_check.sh' in line:
                print(f"   {line}")
    else:
        # Set up cron job to run health check every 6 hours
        cron_command = f"0 */6 * * * /bin/bash {health_check_path}"
        
        if existing_cron:
            new_cron = '\n'.join(existing_cron + [cron_command])
        else:
            new_cron = cron_command
            
        with open('/tmp/crontab.txt', 'w') as f:
            f.write(new_cron + '\n')
            
        !crontab /tmp/crontab.txt
        print(f"✅ Added cron job to run health check every 6 hours: {cron_command}")
except Exception as e:
    print(f"⚠️ Failed to set up cron job: {str(e)}")
    print("You may need to manually set up the cron job with: crontab -e")

## Path Inconsistency Checker

This section scans your scripts for any remaining path inconsistencies that could cause issues.

In [ ]:
# Scan scripts for path inconsistencies
import glob
import re

# Define paths to search
script_paths = [
    os.path.join(workspace_dir, "*.sh"),
    "/src/*.sh",
    "/VisoMaster/*.sh",
    "/VisoMaster/*.py",
    "/vast_ai_provisioning_script.sh"
]

# Patterns to look for
inconsistent_paths = [
    r"/root/VisoMaster",
    r"\./model_assets",  # Relative paths that might be problematic
    r"\./models"
]

print("🔍 Scanning for path inconsistencies...\n")

found_inconsistencies = False

for pattern in script_paths:
    for script_file in glob.glob(pattern, recursive=True):
        try:
            with open(script_file, 'r') as f:
                content = f.read()
                
                for path_pattern in inconsistent_paths:
                    matches = re.findall(path_pattern, content)
                    if matches:
                        found_inconsistencies = True
                        print(f"⚠️ Found inconsistent path in {script_file}:")
                        print(f"   Pattern: {path_pattern}")
                        print(f"   Occurrences: {len(matches)}")
                        
                        # Print line context for each match
                        lines = content.split('\n')
                        for i, line in enumerate(lines):
                            if re.search(path_pattern, line):
                                print(f"   Line {i+1}: {line.strip()}")
                        print()
        except Exception as e:
            print(f"Error reading {script_file}: {str(e)}")

if not found_inconsistencies:
    print("✅ No path inconsistencies found in scanned scripts.")